In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import roc_auc_score

In [29]:
# Total end of March validation
df = pd.read_csv('C:/Users/sgulbin/Work/Analysis/ScoringTesting/korischenko_validation_v2/aggregated_probs_till_march2020.csv')
users = pd.read_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/original_data/users_activated_v7.csv', low_memory = False)
users = users[['login', 'user_ext']]

# Changing "phone" format from int64 to string (object) for successful dataframes merge
dtp.phone = dtp.phone.astype(str)
dtp = pd.merge(dtp, users, left_on = 'phone', right_on = 'login', how = 'left')

# Only guilty and unknown
dtp = dtp.loc[(dtp.Responsibility != 'Фейк')&(dtp.Responsibility != 'Угон')&(dtp.Responsibility != 'Не виновен')]

# Data formatting
dtp['user_ext'] = dtp.user_ext.fillna(0)
df.UserId = df.UserId.astype(int)

# Splitting dtp data into periods for validation
# March
dtp_march = dtp.loc[(dtp.dt >= '2020-03-18') & (dtp.dt <= '2020-03-29')]
dtp_march = dtp_march[['dt','user_ext']]
dtp_march['march'] = 1

# Marking users with accidents
df = pd.merge(df,dtp_march, left_on = 'UserId', right_on = 'user_ext', how = 'left')
df['march'] = df['march'].fillna(0)

# Running roc-auc score calculation
roc_auc_score(df['march'],df.p)

0.8150842258707244

In [ ]:
# Version 2.0 With new drivers
df = pd.read_csv('C:/Users/sgulbin/Work/Analysis/ScoringTesting/korischenko_validation_v2/aggregated_probs_till_march2020.csv')
users = pd.read_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/original_data/users_activated_v7.csv', low_memory = False)
users = users[['login', 'user_ext']]
# All drivers who drove during validation period
new = pd.read_csv('C:/Users/sgulbin/Work/Analysis/ScoringTesting/korischenko_validation_v2/march_validation_period_drivers.csv')
users = pd.merge(new, users, on = 'user_ext', h)

# Changing "phone" format from int64 to string (object) for successful dataframes merge
dtp.phone = dtp.phone.astype(str)
dtp = pd.merge(dtp, users, left_on = 'phone', right_on = 'login', how = 'left')

# Only guilty and unknown
dtp = dtp.loc[(dtp.Responsibility != 'Фейк')&(dtp.Responsibility != 'Угон')&(dtp.Responsibility != 'Не виновен')]

# Data formatting
dtp['user_ext'] = dtp.user_ext.fillna(0)
df.UserId = df.UserId.astype(int)

# Splitting dtp data into periods for validation
# March
dtp_march = dtp.loc[(dtp.dt >= '2020-03-18') & (dtp.dt <= '2020-03-29')]
dtp_march = dtp_march[['dt','user_ext']]
dtp_march['march'] = 1

# Marking users with accidents
df = pd.merge(df,dtp_march, left_on = 'UserId', right_on = 'user_ext', how = 'left')
df['march'] = df['march'].fillna(0)

# Running roc-auc score calculation
roc_auc_score(df['march'],df.p)

In [21]:
# Total October-November validation
dtp = pd.read_csv('C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/accidents_data_for_Korischenko.csv')
df = pd.read_csv('C:/Users/sgulbin/Work/Analysis/ScoringTesting/korischenko_validation/Oct-Nov_validation.csv')

# Data formatting
dtp['user_ext'] = dtp.user_ext.fillna(0)
dtp.user_ext = dtp.user_ext.astype(int)
dtp['date'] = pd.to_datetime(dtp.dt)
# Splitting dtp data into periods for validation
# October-November
dtp_oct_nov = dtp.loc[(dtp.date >= '2019-10-01') & (dtp.date <= '2019-11-30')]
dtp_oct_nov = dtp_oct_nov[['date','user_ext']]
dtp_oct_nov['oct_nov'] = 1
# Marking users with accidents
df = pd.merge(df,dtp_oct_nov, left_on = 'user_ext', right_on = 'user_ext', how = 'left')
df['oct_nov'] = df['oct_nov'].fillna(0)
# Running roc-auc score calculation
for i in range(2,11):
    df1 = df.loc[df['rents_count']>= i]
    print(i,roc_auc_score(df1['oct_nov'],df1.probability))

FileNotFoundError: [Errno 2] File C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/accidents_data_for_Korischenko.csv does not exist: 'C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/accidents_data_for_Korischenko.csv'

In [28]:
# Weekly validation
dtp = pd.read_csv('C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/accidents_data_for_Korischenko.csv')
df = pd.read_csv('C:/Users/sgulbin/Work/Analysis/ScoringTesting/korischenko_validation/Oct-Nov_validation_weekly.csv')

# Data formatting
dtp['user_ext'] = dtp.user_ext.fillna(0)
dtp.user_ext = dtp.user_ext.astype(int)
dtp['date'] = pd.to_datetime(dtp.dt)
# Splitting dtp data into periods for validation
for i in [['week1','2019-10-01','2019-10-07'],
          ['week2','2019-10-08','2019-10-14'],
          ['week3','2019-10-15','2019-10-21'],
          ['week4','2019-10-22','2019-10-28'],
          ['week5','2019-10-29','2019-11-04'],
          ['week6','2019-11-05','2019-11-11'],
          ['week7','2019-11-12','2019-11-18'],
          ['week8','2019-11-19','2019-11-25'],
          ['week9','2019-11-26','2019-11-30'],]:
    dtp[i[0]] = np.where((dtp.date >= i[1])&(dtp.date <= i[2]), 1, 0)

# Marking users with accidents    
df = pd.merge(df, dtp, left_on = 'user_ext', right_on = 'user_ext', how = 'left')
str_cols = ['week1','week2','week3','week4','week5','week6','week7','week8','week9',]
df[str_cols] = df[str_cols].fillna(0)
# Running roc-auc score calculation
for i in [['0','week1'],
          ['1','week2'],
          ['2','week3'],
          ['3','week4'],
          ['4','week5'],
          ['5','week6'],
          ['6','week7'],
          ['7','week8'],
          ['8','week9'],]:
    df1 = df[i].dropna()
    print(i[0],roc_auc_score(df1[i[1]],df1[i[0]]))

FileNotFoundError: [Errno 2] File C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/accidents_data_for_Korischenko.csv does not exist: 'C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/accidents_data_for_Korischenko.csv'

In [26]:
# Upload repairs data
dtp = pd.read_excel('C:/Users/sgulbin/Work/_Выгрузки_и_Расчеты/accidents_4_10_2020.xlsx')
dtp['Дата и время ДТП'] = np.where(
    dtp['Дата и время ДТП']=='4/18/2020 12:00:00 AM', '3/18/2020 12:00:00 AM', dtp['Дата и время ДТП']
)
dtp['Дата и время ДТП'] = np.where(
    dtp['Дата и время ДТП']=='4/20/2020 12:00:00 AM', '3/20/2020 12:00:00 AM', dtp['Дата и время ДТП']
)
dtp['Дата и время ДТП'] = np.where(
    dtp['Дата и время ДТП']=='5/24/2020 12:00:00 AM', '3/24/2020 12:00:00 AM', dtp['Дата и время ДТП']
)
dtp['Дата и время ДТП'] = np.where(
    dtp['Дата и время ДТП']=='7/23/2020 12:00:00 AM', '3/23/2020 12:00:00 AM', dtp['Дата и время ДТП']
)
dtp['Дата и время ДТП'] = pd.to_datetime(dtp['Дата и время ДТП'], dayfirst=False, errors = 'coerce')
dtp.sort_values('Дата и время ДТП', inplace=True)
dtp['Дата и время ДТП'] = pd.to_datetime(dtp['Дата и время ДТП'])
dtp = dtp[dtp['Телефон водителя'].notnull()]
dtp['Телефон водителя'] = dtp['Телефон водителя'].apply(int)
dtp = dtp[dtp['ТС'].notnull()]
dtp.reset_index(drop=True, inplace = True)
dtp['ТС'] = dtp['ТС'].str.replace(' ', '')
dtp = dtp[['Дата и время ДТП', 'Регион','Телефон водителя', 'Виновность', 'ТС', 'Модель автомобиля', 'POOL', 'Тариф','Ремонт не производился','Тип ДТП', 'Вид страхования', 'Пострадавшие','Сумма по заказ-наряду','Выставлено', 'Страховая компания сумма выплаченная']].rename(columns={
    'Дата и время ДТП': 'dt',
    'Регион': 'region',
    'Телефон водителя': 'phone',
    'Виновность': 'Responsibility',
    'ТС': 'license_plate',
    'Модель автомобиля': 'vehicle_model',
    'POOL': 'pool',
    'Тариф': 'tariff',
    'Ремонт не производился': 'no repair conducted',
    'тип ДТП': 'accident type',
    'Вид страхования': 'insurance type',
    'Пострадавшие': 'injured',
    'Сумма по заказ-наряду': 'target_amount',
    'Выставлено': 'Vystavleno',
    'Страховая компания сумма выплаченная': 'insurance paid sum'
})